# BP Novice Judge Allocation Algorithm
Developed by Viran Weerasekera (2017)

In [2]:
## IMPORT JUDGES AND TEAMS

import gurobipy as grb
import pandas as pd
import time


def importRound(roundNumber):
    global draw_csv
    draw_csv = pd.read_csv("draw_r{}.csv".format(roundNumber),index_col=0)
    
def importJudgesMaster():
    ## Import Judges (Master)
    global judgesmaster_csv
    judgesmaster_csv = pd.read_csv("judges_master.csv",index_col=0)
    

    global allJudgeNamesMaster
    global allJudgeStrengthsMaster
    global allJudgeSeensMaster
    global allJudgeECSMaster
    global allJudgeJudgeECSMaster
    global allJudgeIsTraineeMaster
    
    allJudgeNamesMaster = {}
    allJudgeStrengthsMaster = {}
    allJudgeSeensMaster = {}
    allJudgeECSMaster = {}
    allJudgeJudgeECSMaster = {}
    allJudgeIsTraineeMaster = {}

    for ind,row in judgesmaster_csv.iterrows():
        allJudgeNamesMaster[ind] = row[0]
        allJudgeStrengthsMaster[ind] = row[1]
        allJudgeECSMaster[ind] = []
        allJudgeJudgeECSMaster[ind] = []
        if allJudgeStrengthsMaster[ind] <= 1:
            allJudgeIsTraineeMaster = True
        else:
            allJudgeIsTraineeMaster = False
    
def importInit():
    ## Import Judges and Team List
    global judges_csv
    judges_csv = pd.read_csv("judges.csv",index_col=0)
    global teams_csv
    teams_csv = pd.read_csv("teams.csv",index_col=0)
    

    global allJudgeNames
    global allJudgeStrengths
    global allJudgeSeens
    global allJudgeECS
    global allJudgeJudgeECS
    global allJudgeIsTrainee
    
    allJudgeNames = {}
    allJudgeStrengths = {}
    allJudgeSeens = {}
    allJudgeECS = {}
    allJudgeJudgeECS = {}
    allJudgeIsTrainee = {}

    global allTeams
    allTeams = {}
    global allTeamKeys
    allTeamKeys = []


    for ind,row in judges_csv.iterrows():
        allJudgeNames[ind] = row[0]
        allJudgeStrengths[ind] = row[1]
        allJudgeECS[ind] = []
        allJudgeJudgeECS[ind] = []
        if allJudgeStrengths[ind] <= 1:
            allJudgeIsTrainee[ind] = True
        else:
            allJudgeIsTrainee[ind] = False

    for ind,row in teams_csv.iterrows():
        allTeams[ind] = row
        allTeamKeys.append(row[0])


def printJudges():
    for key,value in allJudgeNames.iteritems():
        print("{}: {}".format(key,value))

In [3]:
def addEC(judgeID,teamKey):
    allJudgeECSMaster[judgeID].append(teamKey)
    try:
        allJudgeECS[judgeID].append(teamKey)
    # Throw exception if the judge is not judging in the round
    except Exception:
        print("NB: {} is not judging this round".format(allJudgeNamesMaster[judgeID]))
    
    if teamKey in allTeamKeys:
        print("{} conflicted off {}".format(allJudgeNamesMaster[judgeID],teamKey))
        print("------")
    else:
        print("ERROR: Team {} does not exist".format(teamKey))
        print("------")

In [4]:
def setupSeens():
    global pastJudgesByRound
    pastJudgesByRound = {}
    
    global seenKeys
    seenKeys = []
    
    teams_csv = pd.read_csv("teams.csv",index_col=0)
    
    for ind,row in teams_csv.iterrows():
        pastJudgesByRound[row[0]] = {}
        seenKeys.append(row[0])

def addSeens(rd):
    seen_csv = pd.read_csv("seens_r{}.csv".format(rd),index_col=0)
    for ind,row in seen_csv.iterrows():
        if ind in seenKeys:
            pastJudgesByRound[ind][rd] = []
            for i in range(10)[1:]:
                try:
                    pastJudgesByRound[ind][rd].append(row[i])
                except Exception:
                    pass
        else:
            print("ERROR: Team {} does not have a seen record.".format(ind))

In [5]:
def allocateJudges(rd, minimumPanelAverage,traineeOverride):
    #-------------------------------------------------------------------------------------------------------------------
    venues = {}
    matchups = {}
    roomImportance = {}
    
    number_of_rooms = 0

    # Create a matchup with index equal to the room rank
    # Add past judges to each room
    for index,row in draw_csv.iterrows():
        venues[index] = row[0]
        matchups[index] = [row[1],row[2],row[3],row[4]]
        roomImportance[index] = row[5]
        
    number_of_rooms = len(matchups)
    number_of_judges = len(allJudgeNames)

    #-------------------------------------------------------------------------------------------------------------------
    
    ## Check that there are enough judges - If not, terminate
    allNonTrainees = 0
    for key,value in allJudgeIsTrainee.iteritems():
        if value == False:
            allNonTrainees += 1
            
    if traineeOverride == "false":
        if allNonTrainees >= len(matchups):
            pass
        else:
            print("There is an insufficient number of non-trainee checked in for the number of debates.")
            print("{} non-trainee judges, {} rooms".format(allNonTrainees,len(matchups)))
            return None
    
    
    
    ## Generate Feasible Panels

    fps = {}
    modfps = {}

    start = time.time()
    print("----------")
    print("Generating panels...")

    index = 0
    for i in allJudgeNames.keys():
        if allJudgeStrengths[i] >= minimumPanelAverage:
            if allJudgeIsTrainee[i] and traineeOverride == "false":
                pass
            else:
                index += 1
                fps[index] = [i]
        
        for j in allJudgeNames.keys():
            if i != j:
                if (allJudgeStrengths[i] + allJudgeStrengths[j])/2 >= minimumPanelAverage:
                    if (allJudgeIsTrainee[i] or allJudgeIsTrainee[j]) and traineeOverride == "false":
                        pass
                    else:
                        index += 1
                        fps[index] = [i,j]
                        
            for k in allJudgeNames.keys():
                if i != j and j != k and i != k:
                    if (allJudgeStrengths[i] + allJudgeStrengths[j] + allJudgeStrengths[k])/3 >= minimumPanelAverage:
                        if (allJudgeIsTrainee[i] or allJudgeIsTrainee[j] or allJudgeIsTrainee[k]) and traineeOverride == "false":
                            pass
                        else:
                            index += 1
                            fps[index] = [i,j,k]
    end = time.time()
    print("Panels generated in {:.2f} seconds.".format(end-start))
    print("{} feasible panels".format(len(fps)))
    #-------------------------------------------------------------------------------------------------------------------
    ## SCORE FUNCTION

    scoreFunctions = {}
    start2 = time.time()
    print("----------")
    print("Generating Score Functions for {} panels...".format(len(fps)))

    for panel in fps.keys():
        for room in matchups.keys():
            scoreFunctions[panel,room] = 0

    

    for panelKey,panel in fps.iteritems():
        # Calculate Chair Strength and weight it by 50
        chairStrength = allJudgeStrengths[panel[0]]*50

        # Calculate panel average and weight it by 20
        panelAverage = 0
        factor = 1
        element = 0
        for judge in panel:
            factor += 0.1
            element += 1
            strengthToAdd = factor*allJudgeStrengths[judge]
            strengthToAdd = allJudgeStrengths[judge]
            panelAverage += strengthToAdd
        panelAverage = (panelAverage/element) * 20

        for roomKey in matchups.keys():
            scoreFunctions[panelKey,roomKey] += chairStrength
            scoreFunctions[panelKey,roomKey] += panelAverage

        # Invalidate panels which have trainees on them unless the override is enabled
        
        if traineeOverride == "false":
            for panelKey,panel in fps.iteritems():
                for judge in panel:
                    if allJudgeStrengths[judge] <= 1:
                        for roomKey in matchups.keys():
                            scoreFunctions[panelKey,roomKey] = 0
                            
                            
                            
    # Seen Conflicts
    for panelKey,panel in fps.iteritems():
        for judge in panel:
            for roomKey,room in matchups.iteritems():
                for team in room:
                    if rd >= 2:
                        try:
                            if judge in pastJudgesByRound[team][1]:
                                scoreFunctions[panelKey,roomKey] = scoreFunctions[panelKey,roomKey]/scPenalties[0]
                        except Exception:
                            print("Exception")
                    if rd >= 3:
                        try:
                            if judge in pastJudgesByRound[team][2]:
                                scoreFunctions[panelKey,roomKey] = scoreFunctions[panelKey,roomKey]/scPenalties[1]
                        except Exception:
                            print("Exception")
                    if rd >= 4:
                        try:
                            if judge in pastJudgesByRound[team][3]:
                                scoreFunctions[panelKey,roomKey] = scoreFunctions[panelKey,roomKey]/scPenalties[2]
                        except Exception:
                            print("Exception")
                    if rd >= 5:
                        try:
                            if judge in pastJudgesByRound[team][4]:
                                scoreFunctions[panelKey,roomKey] = scoreFunctions[panelKey,roomKey]/scPenalties[3]
                        except Exception:
                            print("Exception")

    # Explicit Conflicts
    for roomKey,room in matchups.iteritems():
        for panelKey,panel in fps.iteritems():
            for judge in panel:
                if room[0] in allJudgeECS[judge]:
                    scoreFunctions[panelKey,roomKey] = -100000000000
                elif room[1] in allJudgeECS[judge]:
                    scoreFunctions[panelKey,roomKey] = -100000000000
                elif room[2] in allJudgeECS[judge]:
                    scoreFunctions[panelKey,roomKey] = -100000000000
                elif room[3] in allJudgeECS[judge]:
                    scoreFunctions[panelKey,roomKey] = -100000000000

    end2 = time.time()
    print("Score Functions generated in {:.2f} seconds".format(end2-start2))
    #-------------------------------------------------------------------------------------------------------------------
    ## OBJECTIVE FUNCTION

    start3 = time.time()
    print("----------")
    print("Generating Objective Function...")

    a = grb.Model("Allocator")
    a.modelSense = grb.GRB.MAXIMIZE

    x = {}

    for key,value in scoreFunctions.iteritems():
        coefficient = roomImportance[key[1]]*value

        x[key] = a.addVar(vtype = grb.GRB.BINARY, obj = coefficient, name = "x_{}".format(key))

    end3 = time.time()
    print("Objective Function generated in {:.2f} seconds".format(end3-start3))
    #-------------------------------------------------------------------------------------------------------------------

    ## CONSTRAINTS
    start4 = time.time()
    print("----------")
    print("Generating Constraints...")

    # Each panel goes to at most one room
    for key,panel in fps.iteritems():
        a.addConstr(sum(x[key,room] for room in matchups.keys()) <= 1)

    # Each room must receive exactly one panel
    # Relax
    for key,room in matchups.iteritems():
        a.addConstr(sum(x[panel,key] for panel in fps.keys()) == 1)

    # Each judge should go to, at most, one room
    for judge in allJudgeNames.keys():
        affectedPanels = []
        affectedCombos = {}
        for key, panel in fps.iteritems():
            if judge in panel:
                affectedPanels.append(key)
        if allJudgeStrengths[judge] <= 1 and traineeOverride == "false":
            pass
        else:
            a.addConstr(sum(x[p,r] for p in affectedPanels for r in matchups) == 1)


    end4 = time.time()
    print("Constraints generated in {:.2f} seconds".format(end4-start4))
    #-------------------------------------------------------------------------------------------------------------------
    print("----------")
    start5 = time.time()
    a.optimize()
    end5 = time. time()
    print("Optimised in {} seconds".format(end5-start5))
    #-------------------------------------------------------------------------------------------------------------------
    allocatedJudges = []
    print("----------")
    print("Judge Allocation for Round {}".format(rd))
    print("----------")
    for key2,value2 in matchups.iteritems():
        for key,value in fps.iteritems():
            if x[key,key2].x != 0:
                panel = []
                panel.append("{} ©".format(allJudgeNames[value[0]]))
                for e in value[1:]:
                    panel.append(allJudgeNames[e])
                panelString = ", ".join(panel)
                print("Room Rank {} ({}) (Weight {}) (OG {}...) - {}".format(key2,venues[key2],roomImportance[key2],matchups[key2][0][:10],panelString))

                for judge in value:
                    allocatedJudges.append(judge)
                totalStrength = 0
                for judge in value:
                    totalStrength = totalStrength + allJudgeStrengths[judge]
                print("Chair Strength: {}".format(allJudgeStrengths[value[0]]))
                print("Panel Average: {:.1f}".format(totalStrength/len(value)))
                print("----------")

    unallocatedJudges = {}
    print("Unallocated Judges")
    for key,value in allJudgeNames.iteritems():
        if key not in allocatedJudges:
            unallocatedJudges[key] = []
            #for key2,value2 in pastJudges.iteritems():
            #    if key in value2:
            #        unallocatedJudges[key].append(key2)
            avoidArray = []
            for e in unallocatedJudges[key]:
                avoidArray.append(str(e))
            avoidString = ", ".join(avoidArray)
            if avoidString == "":
                print("- {} ({})".format(value,allJudgeStrengths[key]))
            else:
                print("- {} ({}) - Avoid Room {}".format(value,allJudgeStrengths[key],avoidString))
    #-------------------------------------------------------------------------------------------------------------------
    print("----------")   
    print("This Round's Judges")
    allJudgesSC = {}
    for key,value in allJudgeNames.iteritems():
        allJudgesSC[key] = []
        #for key2,value2 in pastJudges.iteritems():
        #    if key in value2:
        #        allJudgesSC[key].append(key2)
        seenArray = []
        for e in allJudgesSC[key]:
            seenArray.append(str(e))
        seenString = ", ".join(seenArray)
        ecString = ", ".join(allJudgeECS[key])
        
        print("- {} ({}) - SC {}, EC {}".format(value, allJudgeStrengths[key],seenString, ecString))
    
    #-------------------------------------------------------------------------------------------------------------------

<strong>Functions</strong>:<br>
importInit()<br>
importRound(roundNumber)<br>
addEC(judgeID,TeamKey)<br>
allocateJudges(rd, minimumPanelAverage,traineeOverride)

In [6]:
# Initial Import
importInit()
importJudgesMaster()

# Conflicts

In [8]:
scPenalties = [1,1]

setupSeens()
#addSeens(1)

In [10]:
importRound(1)
allocateJudges(1,3,"false")

----------
Generating panels...
Panels generated in 0.01 seconds.
2799 feasible panels
----------
Generating Score Functions for 2799 panels...
Score Functions generated in 0.39 seconds
----------
Generating Objective Function...
Objective Function generated in 0.29 seconds
----------
Generating Constraints...
Constraints generated in 0.60 seconds
----------
Optimize a model with 2823 rows, 25191 columns and 124047 nonzeros
Variable types: 0 continuous, 25191 integer (25191 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+12]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2799 rows and 20340 columns
Presolve time: 0.25s
Presolved: 24 rows, 4851 columns, 18351 nonzeros
Variable types: 0 continuous, 4851 integer (4851 binary)

Root relaxation: objective 3.870000e+04, 162 iterations, 0.01 seconds

 

In [79]:
# Add a method that checks each team in the room - If the team names don't match up, throw an error